## Import Package

In [7]:
# required packages
import numpy as np
import pandas as pd
import numba as nb
import warnings
import gc

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# display set up
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 3)
pd.options.display.max_rows = 999

## Load Data

In [8]:
# Functions for memory reduction
@nb.jit()
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return(df)

In [9]:
%%time

prop_2016 = pd.read_csv('data/prop_2016.csv')
prop_2016 = reduce_mem_usage(prop_2016)

prop_2017 = pd.read_csv('data/prop_2017.csv')
prop_2017 = reduce_mem_usage(prop_2017)

train = pd.read_csv('data/train.csv')
train = reduce_mem_usage(train)

Mem. usage decreased to 1022.05 Mb (75.1% reduction)
Mem. usage decreased to 1022.05 Mb (75.1% reduction)
Mem. usage decreased to 58.76 Mb (75.2% reduction)
CPU times: user 4min 10s, sys: 3min 3s, total: 7min 14s
Wall time: 8min 8s


## Modeling Prep

In [10]:
def encoding(train, prop_2016, prop_2017, categorical_features):
    """
    To limit the value to small numbers since values in categorical_feature is suggested to be small.
    Now the number denoting missing values will be 0 instead of -1.
    """
    from category_encoders import BinaryEncoder
    
    split_index = [len(train), len(prop_2016), len(prop_2017)]
    
    
    prop = pd.concat([train[categorical_features],
                      prop_2016[categorical_features], 
                      prop_2017[categorical_features]], ignore_index=True)
    
    encoder = BinaryEncoder(cols=categorical_list, drop_invariant = True)
    transformed = encoder.fit_transform(prop)
    train_be = transformed.iloc[: split_index[0]]
    prop_2016_be = transformed.iloc[split_index[0] : split_index[0]+split_index[1]]
    prop_2017_be = transformed.iloc[split_index[0]+split_index[1] : ]
    

    train = pd.concat([train.drop(columns=categorical_features), train_be], axis = 1)
    prop_2016 = pd.concat([prop_2016.drop(columns=categorical_features), prop_2016_be], axis = 1)
    prop_2017 = pd.concat([prop_2017.drop(columns=categorical_features), prop_2017_be], axis = 1)

    
    del prop, transformed, train_be, prop_2016_be, prop_2017_be
    gc.collect()
    return(train, prop_2016, prop_2017)

# Do encoding for cateogircal features
categorical_features = ['airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid', 'buildingqualitytypeid',
              'fips', 'heatingorsystemtypeid', 'propertycountylandusecode', 'propertylandusetypeid',
              'propertyzoningdesc', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip',
              'storytypeid', 'typeconstructiontypeid', 'geo_cluster']

train, prop_2016, prop_2017 = encoding(train, prop_2016, prop_2017, categorical_features)

NameError: name 'categorical_list' is not defined

In [ ]:
# Transform to Numpy matrices
X = train.drop(columns='logerror')
y = train.logerror.values

# Specify feature names
feature_names = [col for col in X.columns]

# Get categorical features
categorical_indices = []
for i, n in enumerate(X.columns):
    if n in categorical_features:
        categorical_indices.append(i)

# Perform shuffled train/test split
# np.random.seed(910)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Keep outlier values (roughly outside 99% percentile) out of the training dataset
# outlier_threshold = 0.4
# mask = (abs(y_train) <= outlier_threshold)
# X_train = X_train.iloc[mask, :]
# y_train = y_train[mask]

print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_val shape: {}".format(X_test.shape))
print("y_val shape: {}".format(y_test.shape))

## Hyperparameter Tuning

We will conduct bayesian hyperparameter tuning with the help of the package `HyperOpt`

In [ ]:
params = {
    "n_estimators": Categorical([100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]),
    "max_depth":  Integer(1, 100),
    "min_samples_leaf": Integer(1, 10),
    "min_samples_split": Integer(1, 15),
    "criterion": Categorical(["gini", "entropy"]),
    "bootstrap": Categorical([True, False]),
}


clf = RandomForestRegressor()


# define the search
search = BayesSearchCV(
    estimator=clf,
    search_spaces=params,
    n_jobs=-1,
    cv=10,
    n_iter=300,
    scoring="accuracy",
    verbose=1
)


# perform the search
search.fit(X_train, y_train)

# report the best result
print(search.best_score_)
print(search.best_params_)

In [ ]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

space = {
    # choose within the list
    "n_estimators": hp.choice("n_estimators", [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]),
    # quantized uniform distribution to prevent float value
    "max_depth": hp.quniform("max_depth", 1, 100, 5),
    "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 5, 1),
    "min_samples_split": hp.quniform("min_samples_split", 1, 15, 1),
    # choose within the list
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
    # Whether or not to bootstrap
    "bootstrap": hp.choice("bootstrap", [True, False]),
}

In [ ]:
# define custom scoring function similar to the kaggle evaluation
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

def my_custom_loss_func(y_true, y_pred):
    diff = np.log1p(y_true) - np.log1p(y_pred)
    return diff**2

scorer = make_scorer(my_custom_loss_func, greater_is_better=False)


# define objective function
def hyperparameter_tuning(params):
    clf = RandomForestRegressor(**params,n_jobs=-1)
    loss = cross_val_score(clf, X_train, y_train ,scoring=scorer).mean()
    return {"loss": loss, "status": STATUS_OK} ## hyperparameter minimizes 

In [ ]:
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials
)

print("Best: {}".format(best))

In [ ]:
trials.results

In [ ]:
# CatBoost parameters Setting
params = {'loss_functiomn': 'MAE',
         'eval_metric': 'MAE',
         'nan_mode': 'Min',
         'iterations': 1000,
         'l2_leaf_reg': 5}

In [ ]:
%%time
# Train CatBoost Regressor with cross-validated early-stopping
test_pool = Pool(X_test, y_test.astype(np.float64), cat_features=categorical_indices)

np.random.seed(910)
model = CatBoostRegressor(**params)
model.fit(X_train, y_train,
          cat_features=categorical_indices,
          use_best_model=True, eval_set=test_pool, early_stopping_rounds=50, verbose=False)


# Evaluate on train and validation sets
print(f"Train score: {abs(model.predict(X_train) - y_train).mean() * 100}")
print(f"Test score: {abs(model.predict(X_test) - y_test).mean() * 100}")

## Training on all Data

In [ ]:
# Remove outlier (if any)
outlier_threshold = 0.4
mask = (abs(y) <= outlier_threshold)
X = X.iloc[mask, :]
y = y[mask]

# Train multiple models
bags = 8
models = []
params['iterations'] = 1000
for i in range(bags):
    print("Start training model {}".format(i))
    params['random_seed'] = i
    np.random.seed(910)
    model = CatBoostRegressor(**params)
    model.fit(X, y, cat_features=categorical_indices, verbose=False)
    models.append(model)
    
# Sanity check (make sure scores on a small portion of the dataset are reasonable)
for i, model in enumerate(models):
    print("model {}: {}".format(i, abs(model.predict(X_test) - y_test).mean() * 100))

In [ ]:
def transform_test_features(prop_2016, prop_2017):
    """
    Helper method that prepares prop_2016 and prop_2017 for prediction.
    """
    prop_2016 = drop_features(prop_2016)
    prop_2017 = drop_features(prop_2017)
    
    # Create three datetime columns that does not exist in those dataset
    prop_2016['year'] = 0
    prop_2017['year'] = 1
    
    prop_2016['month'] = 8 # randomly select one month
    prop_2017['month'] = 8 # randomly select one month
    
    prop_2016['quarter'] = 3 # randomly select one quarter
    prop_2017['quarter'] = 3 # randomly select one quarter
    
    # Reorder to maintain categorical indices
    prop_2016 = prop_2016[['year', 'month', 'quarter'] + list(prop_2016.columns[:-3])]
    prop_2017 = prop_2017[['year', 'month', 'quarter'] + list(prop_2017.columns[:-3])]
    return(prop_2016, prop_2017)

def predict_and_export(models, prop_2016, prop_2017, file_name):
    """
    Helper method to make predicition and export results to csv.
    """
    # Construct DataFrame for prediction results
    submission_2016 = pd.DataFrame()
    submission_2017 = pd.DataFrame()
    submission_2016['ParcelId'] = prop_2016.parcelid
    submission_2017['ParcelId'] = prop_2017.parcelid
    
    # Prepare dataset for prediction
    prop_2016, prop_2017 = transform_test_features(prop_2016, prop_2017)
    
    # Make Prediction across multiple models
    pred_2016, pred_2017 = [], []
    for i, model in enumerate(models):
        print("Start model {} (2016)".format(i))
        pred_2016.append(model.predict(prop_2016))
        print("Start model {} (2017)".format(i))
        pred_2017.append(model.predict(prop_2017))
    
    # Take average across all models
    mean_pred_2016 = np.mean(pred_2016, axis=0)
    mean_pred_2017 = np.mean(pred_2017, axis=0)
    
    # Formatting for submission
    submission_2016['201610'] = [float(format(x, '.4f')) for x in mean_pred_2016]
    submission_2016['201611'] = submission_2016['201610']
    submission_2016['201612'] = submission_2016['201610']

    submission_2017['201710'] = [float(format(x, '.4f')) for x in mean_pred_2017]
    submission_2017['201711'] = submission_2017['201710']
    submission_2017['201712'] = submission_2017['201710']
    
    submission = pd.merge(submission_2016,submission_2017, how='inner', on='ParcelId')
    submission.to_csv(file_name, index=False)
    print("Submission Successfully Created.")
    return(submission, pred_2016, pred_2017)

In [ ]:
%%time
del train
gc.collect()

file_name = 'submission/final_cat.csv'
submission, pred_2016, pred_2017 = predict_and_export(models, prop_2016, prop_2017, file_name)